<a href="https://colab.research.google.com/github/Aryan-Deshpande/NestedML/blob/master/linear_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#torch architecture imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.datasets import ImageFolder
import torchvision.transforms as t
from torchvision import transforms, datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
trans = transforms.Compose([
    transforms.ToTensor(),
])

image = Image.open('/content/test.png').convert('L')
image = trans(image)
print(image.shape, " aft trans")
# open image dataset
##load = torchvision.datasets.ImageFolder(root='C://Users//deshp//Desktop//Dp//src//pytorch//test1//test1')
##testing = DataLoader(batch_size=64, shuffle=True, dataset=load)

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.encoder = nn.Sequential(
            # reduce input image size
            nn.Linear(100*96, 3200),
            nn.ReLU(),

            nn.Linear(3200, 512),
            nn.ReLU(),

            nn.Linear(512, 128),
            nn.ReLU(),

            nn.Linear(128, 64),
            nn.ReLU(),

            nn.Linear(64, 12),
            nn.ReLU(),

            nn.Linear(12, 3),
            nn.ReLU(),
        )

        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(),

            nn.Linear(12, 64),
            nn.ReLU(),

            nn.Linear(64, 128),
            nn.ReLU(),

            nn.Linear(128, 512),
            nn.ReLU(),      

            nn.Linear(512, 3200),
            nn.ReLU(), 

            nn.Linear(3200, 100*96),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)

        decoded = self.decoder(encoded)

        #print(decoded)

        return decoded

model = Autoencoder()
print(model.encoder[0].weight.shape)

# training
# hyperparameters

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)
crit = nn.MSELoss()
epochs = 100

# viewing architecture
from torch.utils.tensorboard import SummaryWriter

logs = 'logs/'
writer = SummaryWriter(logs)


output = []
for epoch in range(100000):
    x = image.view(image.size(0), -1)
    recons = model(x)

    #print(recons)

    if epoch % 10 == 0:    
        output.append(recons)

    loss = crit(recons, x)

    loss.backward()
    writer.add_scalar('training_loss', loss.item(), epoch)
    print(f"{loss.item()}")

    optimizer.step()
    optimizer.zero_grad()

writer.add_graph(model, input_to_model=image.view( image.size(0),-1) )

print(output)
# loss in the neural network architecture is not good.

for epoch in range(10):
    plt.figure(figsize=(9,2))
    plt.gray()

    recon = output[epoch].detach().numpy()

    for i,item in enumerate(recon):
        if i >= 9: break
        plt.subplot(2,9,9+i+1)
        item = item.reshape(-1,100,96)

        plt.imshow(item[0])





Streaming output truncated to the last 5000 lines.
3.927000580006279e-05
5.8406294556334615e-05
4.173739944235422e-05
5.4004434787202626e-05
4.478081609704532e-05
5.005694765713997e-05
4.794285632669926e-05
4.694768358604051e-05
5.0783604820026085e-05
4.472848740988411e-05
5.303210855345242e-05
4.327861461206339e-05
5.459676685859449e-05
4.2417566874064505e-05
5.553942173719406e-05
4.1969469748437405e-05
5.6013032008195296e-05
4.177115988568403e-05
5.619986040983349e-05
4.16977381973993e-05
5.62680943403393e-05
4.164451456745155e-05
5.637332651531324e-05
4.1524785046931356e-05
5.664940908900462e-05
4.127372812945396e-05
5.7195611589122564e-05
4.084183819941245e-05
5.8133024140261114e-05
4.0178416384151205e-05
5.9591358876787126e-05
3.9266422390937805e-05
6.172627035994083e-05
3.8108024455141276e-05
6.473274697782472e-05
3.6765635741176084e-05
6.883627065690234e-05
3.537565498845652e-05
7.416569860652089e-05
3.418754931772128e-05
8.049958705669269e-05
3.346563607919961e-05
8.68385686771

In [ ]:
torch.save(model, "autoencversion1.pt")